# INVA - AI 모의면접 테스트

## 1. 환경 설정

In [ ]:
#코랩환경에서 실행
!pip install google-genai

In [ ]:
import json
import re
import uuid
from typing import Optional
from enum import Enum
from dataclasses import dataclass

from google import genai
from google.genai import types

GEMINI_API_KEY = ""  # 실제 키로 교체

# 클라이언트 생성
client = genai.Client(api_key=GEMINI_API_KEY)

print("✅ Gemini API 설정 완료 (새 SDK)")

## 2. 데이터 모델 정의

In [ ]:
class QuestionCategory(str, Enum):
    """질문 카테고리"""
    MOTIVATION = "지원동기"
    PROJECT_SUCCESS = "프로젝트 경험"
    TECHNICAL = "기술/역량"
    COLLABORATION = "협업/커뮤니케이션"
    PROBLEM_SOLVING = "문제해결"
    FAILURE_OVERCOME = "실패극복"
    GROWTH = "성장가능성"


class Grade(str, Enum):
    """평가 등급"""
    S = "S"
    A = "A"
    B = "B"
    C = "C"
    D = "D"


@dataclass
class Question:
    """면접 질문"""
    id: int
    category: str
    content: str


@dataclass
class Answer:
    """사용자 답변"""
    question_id: int
    content: str
    duration_seconds: Optional[int] = None


@dataclass
class STARScore:
    """STAR 기법 점수 (각 1-10점)"""
    situation: int
    task: int
    action: int
    result: int
    
    @property
    def total(self) -> int:
        return self.situation + self.task + self.action + self.result


@dataclass
class AdditionalScore:
    """추가 평가 점수 (각 1-10점)"""
    logic: int           # 논리성
    specificity: int     # 구체성
    job_relevance: int   # 직무관련성
    time_balance: int    # 시간분배
    
    @property
    def total(self) -> int:
        return self.logic + self.specificity + self.job_relevance + self.time_balance


@dataclass
class QuestionFeedback:
    """질문별 상세 피드백"""
    question_id: int
    question_content: str
    user_answer: str
    star_score: STARScore
    additional_score: AdditionalScore
    total_score: float
    strengths: list
    improvements: list
    example_answer: str


print("✅ 데이터 모델 정의 완료")

## 3. AI 서비스 클래스 (새 SDK 버전)

In [ ]:
class InterviewAIService:
    """모의면접 AI 서비스 (google-genai SDK 사용)"""
    
    def __init__(self, client: genai.Client, model_name: str = "gemini-2.5-flash-lite"):
        self.client = client
        self.model_name = model_name
        self._sessions = {}
    
    def _generate(self, prompt: str) -> str:
        """Gemini API 호출"""
        response = self.client.models.generate_content(
            model=self.model_name,
            contents=prompt
        )
        return response.text
    
    # ============================================
    # 질문 생성
    # ============================================
    
    def generate_questions(self, resume_text: str, job_position: str) -> tuple:
        """자기소개서 기반 7개 맞춤형 면접 질문 생성"""
        prompt = self._build_question_prompt(resume_text, job_position)
        
        response_text = self._generate(prompt)
        questions = self._parse_questions(response_text)
        
        interview_id = str(uuid.uuid4())
        self._sessions[interview_id] = {
            "job_position": job_position,
            "resume_text": resume_text,
            "questions": questions
        }
        
        return interview_id, questions
    
    def _build_question_prompt(self, resume_text: str, job_position: str) -> str:
        return f"""당신은 {job_position} 포지션 면접관입니다.
아래 자기소개서를 읽고, 실제 면접에서 물어볼 법한 심층 질문 7개를 생성하세요.

## 질문 생성 규칙
1. 모든 질문은 서로 다른 주제/카테고리여야 합니다.
2. 자기소개서 내용을 구체적으로 언급하며 질문하세요.
3. 구체적인 경험, 상황, 행동, 결과를 자연스럽게 물어보는 심층 질문으로 작성하세요.
4. 단답형이 아닌 구술형 답변을 유도하는 질문으로 작성하세요.
5. 질문에 'STAR 기법', 'STAR 방식' 등 특정 답변 형식을 요구하는 표현을 절대 사용하지 마세요.

## 필수 포함 카테고리 (7개)
1. 지원동기 - 해당 직무/회사에 대한 관심과 이해도
2. 프로젝트 경험 - 성공적으로 완수한 프로젝트 경험
3. 기술/역량 - 직무 관련 기술적 역량 심화
4. 협업/커뮤니케이션 - 팀워크, 갈등 해결 경험
5. 문제해결 - 어려운 상황을 해결한 경험
6. 실패극복 - 실패/약점을 극복한 사례
7. 성장가능성 - 미래 계획, 자기개발 의지

## 자기소개서
{resume_text}

## 지원 직무
{job_position}

## 출력 형식 (반드시 아래 JSON 형식으로만 출력하세요)
```json
[
  {{"id": 1, "category": "지원동기", "content": "질문 내용..."}},
  {{"id": 2, "category": "프로젝트 경험", "content": "질문 내용..."}},
  {{"id": 3, "category": "기술/역량", "content": "질문 내용..."}},
  {{"id": 4, "category": "협업/커뮤니케이션", "content": "질문 내용..."}},
  {{"id": 5, "category": "문제해결", "content": "질문 내용..."}},
  {{"id": 6, "category": "실패극복", "content": "질문 내용..."}},
  {{"id": 7, "category": "성장가능성", "content": "질문 내용..."}}
]
```
"""

    def _parse_questions(self, response_text: str) -> list:
        json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            json_match = re.search(r'\[.*\]', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
            else:
                raise ValueError(f"질문 생성 결과를 파싱할 수 없습니다: {response_text[:200]}")
        
        questions_data = json.loads(json_str)
        return [Question(id=q["id"], category=q["category"], content=q["content"]) for q in questions_data]

    # ============================================
    # 답변 평가
    # ============================================
    
    def evaluate_answers(self, interview_id: str, answers: list) -> dict:
        """7개 답변에 대한 종합 평가 수행"""
        session = self._sessions.get(interview_id)
        if not session:
            raise ValueError(f"세션을 찾을 수 없습니다: {interview_id}")
        
        job_position = session["job_position"]
        questions = session["questions"]
        
        question_feedbacks = []
        for i, answer in enumerate(answers):
            print(f"  평가 중... ({i+1}/7)")
            question = next(q for q in questions if q.id == answer.question_id)
            feedback = self._evaluate_single_answer(question, answer, job_position)
            question_feedbacks.append(feedback)
        
        return self._generate_comprehensive_feedback(
            interview_id, job_position, question_feedbacks
        )

    def _evaluate_single_answer(self, question: Question, answer: Answer, job_position: str) -> QuestionFeedback:
        """단일 답변 평가"""
        prompt = f"""당신은 {job_position} 채용 면접 평가 전문가입니다.
아래 면접 질문과 지원자의 답변을 STAR 기법 기준으로 상세히 평가하세요.

## 평가 기준 (각 항목 1-10점)

### STAR 기법 평가
- Situation (상황): 상황과 배경 설명의 명확성
- Task (과제): 본인의 역할과 목표 정의의 명확성
- Action (행동): 구체적인 행동과 노력의 상세함
- Result (결과): 성과와 학습 포인트의 구체성

### 추가 평가
- 논리성: 답변 구조의 논리적 흐름과 일관성
- 구체성: 수치, 사례, 예시의 구체적 제시
- 직무관련성: {job_position} 직무와의 연관성
- 시간분배: 답변 길이의 적절성 (이상적: 1-2분, 150-300자)

## 질문
카테고리: {question.category}
내용: {question.content}

## 지원자 답변
{answer.content}

## 출력 형식 (반드시 아래 JSON 형식으로만 출력하세요)
```json
{{
  "star_score": {{
    "situation": 7,
    "task": 6,
    "action": 8,
    "result": 5
  }},
  "additional_score": {{
    "logic": 7,
    "specificity": 6,
    "job_relevance": 8,
    "time_balance": 7
  }},
  "strengths": [
    "잘한 점 1",
    "잘한 점 2"
  ],
  "improvements": [
    "개선 포인트 1",
    "개선 포인트 2"
  ],
  "example_answer": "이 질문에 대한 모범 답변 예시 (200자 내외)"
}}
```
"""
        
        response_text = self._generate(prompt)
        
        # JSON 파싱
        json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            json_str = json_match.group(0) if json_match else "{}"
        
        data = json.loads(json_str)
        
        star_score = STARScore(**data["star_score"])
        additional_score = AdditionalScore(**data["additional_score"])
        
        # 총점 계산 (80점 만점 -> 100점 환산)
        raw_total = star_score.total + additional_score.total
        total_score = round((raw_total / 80) * 100, 1)
        
        return QuestionFeedback(
            question_id=question.id,
            question_content=question.content,
            user_answer=answer.content,
            star_score=star_score,
            additional_score=additional_score,
            total_score=total_score,
            strengths=data["strengths"],
            improvements=data["improvements"],
            example_answer=data["example_answer"]
        )

    def _calculate_grade(self, score: float) -> str:
        if score >= 90: return "S"
        elif score >= 80: return "A"
        elif score >= 70: return "B"
        elif score >= 60: return "C"
        else: return "D"

    def _generate_comprehensive_feedback(self, interview_id: str, job_position: str, feedbacks: list) -> dict:
        """종합 피드백 생성"""
        overall_score = round(sum(f.total_score for f in feedbacks) / len(feedbacks), 1)
        overall_grade = self._calculate_grade(overall_score)
        
        star_averages = {
            "situation": round(sum(f.star_score.situation for f in feedbacks) / 7, 1),
            "task": round(sum(f.star_score.task for f in feedbacks) / 7, 1),
            "action": round(sum(f.star_score.action for f in feedbacks) / 7, 1),
            "result": round(sum(f.star_score.result for f in feedbacks) / 7, 1),
        }
        
        additional_averages = {
            "logic": round(sum(f.additional_score.logic for f in feedbacks) / 7, 1),
            "specificity": round(sum(f.additional_score.specificity for f in feedbacks) / 7, 1),
            "job_relevance": round(sum(f.additional_score.job_relevance for f in feedbacks) / 7, 1),
            "time_balance": round(sum(f.additional_score.time_balance for f in feedbacks) / 7, 1),
        }
        
        # 종합 조언 생성
        print("  종합 피드백 생성 중...")
        summary_prompt = self._build_summary_prompt(job_position, feedbacks, star_averages, additional_averages, overall_score)
        summary_response = self._generate(summary_prompt)
        
        json_match = re.search(r'```json\s*(.*?)\s*```', summary_response, re.DOTALL)
        if json_match:
            summary_data = json.loads(json_match.group(1))
        else:
            summary_data = {
                "overall_strengths": ["전반적으로 성실한 답변"],
                "overall_improvements": ["구체적인 경험 제시 필요"],
                "final_advice": "면접 준비를 더 체계적으로 진행하세요."
            }
        
        return {
            "interview_id": interview_id,
            "job_position": job_position,
            "overall_score": overall_score,
            "overall_grade": overall_grade,
            "star_averages": star_averages,
            "additional_averages": additional_averages,
            "question_feedbacks": feedbacks,
            "overall_strengths": summary_data["overall_strengths"],
            "overall_improvements": summary_data["overall_improvements"],
            "final_advice": summary_data["final_advice"]
        }

    def _build_summary_prompt(self, job_position, feedbacks, star_avg, add_avg, score):
        feedback_summary = "\n".join([
            f"Q{f.question_id}. 점수: {f.total_score}점, 강점: {', '.join(f.strengths[:2])}, 개선점: {', '.join(f.improvements[:2])}"
            for f in feedbacks
        ])
        
        return f"""당신은 {job_position} 채용 면접 평가 전문가입니다.
지원자의 7개 면접 답변 평가 결과를 종합하여 최종 피드백을 작성하세요.

## 평가 결과 요약
- 종합 점수: {score}점

### STAR 항목별 평균
- Situation: {star_avg['situation']}점 / Task: {star_avg['task']}점 / Action: {star_avg['action']}점 / Result: {star_avg['result']}점

### 추가 평가 항목별 평균
- 논리성: {add_avg['logic']}점 / 구체성: {add_avg['specificity']}점 / 직무관련성: {add_avg['job_relevance']}점 / 시간분배: {add_avg['time_balance']}점

### 질문별 요약
{feedback_summary}

## 출력 형식 (반드시 아래 JSON 형식으로만 출력하세요)
```json
{{
  "overall_strengths": ["전반적인 강점 1", "전반적인 강점 2", "전반적인 강점 3"],
  "overall_improvements": ["전반적인 개선점 1 (구체적 방법 포함)", "전반적인 개선점 2", "전반적인 개선점 3"],
  "final_advice": "{job_position} 직무 준비를 위한 200자 내외 종합 조언"
}}
```
"""


print("✅ InterviewAIService 클래스 정의 완료 (새 SDK)")

## 4. 테스트 실행

In [ ]:
# AI 서비스 인스턴스 생성
# 모델 옵션: gemini-2.5-flash-lite (가장 빠름), gemini-2.5-flash, gemini-2.0-flash
ai_service = InterviewAIService(client, model_name="gemini-2.5-flash-lite")
print("✅ AI 서비스 초기화 완료")
print(f"   사용 모델: {ai_service.model_name}")

### 4.1 자기소개서 & 직무 입력

In [ ]:
# 📝 자기소개서 입력
print("="*60)
print("📄 자기소개서를 입력해주세요")
print("="*60)
print("(여러 줄 입력 가능, 입력이 끝나면 빈 줄에서 Enter 두 번)\n")

lines = []
empty_count = 0
while True:
    line = input()
    if line == "":
        empty_count += 1
        if empty_count >= 2:
            break
        lines.append(line)
    else:
        empty_count = 0
        lines.append(line)

resume_text = "\n".join(lines).strip()

print("\n" + "="*60)
print(f"✅ 자기소개서 입력 완료 ({len(resume_text)}자)")
print("="*60)

In [ ]:
# 💼 지원 직무 입력
print("\n💼 지원 직무를 입력해주세요 (예: 프론트엔드 개발자, 백엔드 개발자, 데이터 분석가 등)")
job_position = input("지원 직무: ").strip()

print(f"\n✅ 지원 직무: {job_position}")

### 4.2 면접 질문 생성

In [ ]:
print("🤖 면접 질문 생성 중...")
interview_id, questions = ai_service.generate_questions(resume_text, job_position)

print(f"\n✅ 면접 ID: {interview_id}")
print("="*60)
print("📋 생성된 면접 질문 (7개)")
print("="*60)

for q in questions:
    print(f"\n[Q{q.id}] ({q.category})")
    print(f"   {q.content}")

### 4.3 답변 입력

In [ ]:
# 📝 각 질문에 대한 답변 입력
print("="*60)
print("📝 각 질문에 대한 답변을 입력해주세요")
print("="*60)
print("(각 답변은 여러 줄 입력 가능, 입력이 끝나면 빈 줄에서 Enter 두 번)\n")

answers = []

for q in questions:
    print(f"\n{'─'*60}")
    print(f"[Q{q.id}] ({q.category})")
    print(f"{q.content}")
    print(f"{'─'*60}")
    print("답변 입력:")
    
    lines = []
    empty_count = 0
    while True:
        line = input()
        if line == "":
            empty_count += 1
            if empty_count >= 2:
                break
            lines.append(line)
        else:
            empty_count = 0
            lines.append(line)
    
    answer_content = "\n".join(lines).strip()
    answers.append(Answer(question_id=q.id, content=answer_content))
    print(f"✅ Q{q.id} 답변 저장 완료 ({len(answer_content)}자)")

print(f"\n{'='*60}")
print(f"✅ 총 {len(answers)}개 답변 입력 완료!")
print(f"{'='*60}")

### 4.4 답변 평가 및 피드백 생성

In [ ]:
print("🤖 답변 평가 중...")
feedback = ai_service.evaluate_answers(interview_id, answers)
print("\n✅ 평가 완료!")

## 5. 결과 확인

### 5.1 종합 점수

In [ ]:
print("="*60)
print("📊 종합 평가 결과")
print("="*60)
print(f"\n🎯 종합 점수: {feedback['overall_score']}점")
print(f"🏆 등급: {feedback['overall_grade']}")

print("\n📈 STAR 항목별 평균 (10점 만점):")
for key, value in feedback['star_averages'].items():
    bar = "█" * int(value) + "░" * (10 - int(value))
    print(f"   {key:12}: {bar} {value}점")

print("\n📈 추가 평가 항목별 평균 (10점 만점):")
labels = {"logic": "논리성", "specificity": "구체성", "job_relevance": "직무관련성", "time_balance": "시간분배"}
for key, value in feedback['additional_averages'].items():
    bar = "█" * int(value) + "░" * (10 - int(value))
    print(f"   {labels[key]:12}: {bar} {value}점")

### 5.2 질문별 상세 피드백

In [ ]:
print("="*60)
print("📝 질문별 상세 피드백")
print("="*60)

for f in feedback['question_feedbacks']:
    print(f"\n{'─'*60}")
    print(f"[Q{f.question_id}] {f.question_content[:50]}...")
    print(f"{'─'*60}")
    
    print(f"\n📊 점수: {f.total_score}점")
    
    print(f"\n   STAR 점수:")
    print(f"   - Situation: {f.star_score.situation}/10")
    print(f"   - Task: {f.star_score.task}/10")
    print(f"   - Action: {f.star_score.action}/10")
    print(f"   - Result: {f.star_score.result}/10")
    
    print(f"\n   추가 평가:")
    print(f"   - 논리성: {f.additional_score.logic}/10")
    print(f"   - 구체성: {f.additional_score.specificity}/10")
    print(f"   - 직무관련성: {f.additional_score.job_relevance}/10")
    print(f"   - 시간분배: {f.additional_score.time_balance}/10")
    
    print(f"\n✅ 잘한 점:")
    for s in f.strengths:
        print(f"   • {s}")
    
    print(f"\n⚠️ 개선 포인트:")
    for i in f.improvements:
        print(f"   • {i}")
    
    print(f"\n💡 모범 답변 예시:")
    print(f"   {f.example_answer}")

### 5.3 종합 피드백

In [ ]:
print("="*60)
print("🎯 종합 피드백")
print("="*60)

print("\n✅ 전반적인 강점:")
for s in feedback['overall_strengths']:
    print(f"   • {s}")

print("\n⚠️ 전반적인 개선점:")
for i in feedback['overall_improvements']:
    print(f"   • {i}")

print("\n💬 최종 조언:")
print(f"   {feedback['final_advice']}")

---
## 6. 전체 결과 JSON 출력 (디버깅용)

In [ ]:
from dataclasses import asdict

# QuestionFeedback를 dict로 변환
def feedback_to_dict(f):
    return {
        "question_id": f.question_id,
        "question_content": f.question_content,
        "user_answer": f.user_answer,
        "star_score": asdict(f.star_score),
        "additional_score": asdict(f.additional_score),
        "total_score": f.total_score,
        "strengths": f.strengths,
        "improvements": f.improvements,
        "example_answer": f.example_answer
    }

output = {
    "interview_id": feedback["interview_id"],
    "job_position": feedback["job_position"],
    "overall_score": feedback["overall_score"],
    "overall_grade": feedback["overall_grade"],
    "star_averages": feedback["star_averages"],
    "additional_averages": feedback["additional_averages"],
    "question_feedbacks": [feedback_to_dict(f) for f in feedback["question_feedbacks"]],
    "overall_strengths": feedback["overall_strengths"],
    "overall_improvements": feedback["overall_improvements"],
    "final_advice": feedback["final_advice"]
}

print(json.dumps(output, ensure_ascii=False, indent=2))